In [ ]:
# !pip install pandas
# !pip install py7zr
# !pip install openpyxl

In [1]:
import os
import pandas as pd
import datetime
import py7zr
import openpyxl

In [2]:
zipfilename = './20200701.7z'
files = py7zr.SevenZipFile(zipfilename,'r')
files.extractall(path = '')
files.close()

In [3]:
path = zipfilename[2:10]
def read_file(path):
    mm = [m for m,k,w in os.walk(path)]
    kk = [k for m,k,w in os.walk(path)]
    ww = [w for m,k,w in os.walk(path)]
    file_path_1 = []
    file_path_2 = []
    for i in range(1,len(mm)):
        path_1 = mm[i] + '/' + ww[i][0]
        path_2 = mm[i] + '/' + ww[i][1]
        file_path_1.append(path_1)
        file_path_2.append(path_2)
    return file_path_1,file_path_2

In [4]:
file_path_1,file_path_2 = read_file(path)

In [5]:
print(file_path_1)
print('==================')
print(file_path_2)

['20200701\\1/20200701.csv', '20200701\\2/20200701.csv', '20200701\\3/20200701.csv']
['20200701\\1/log_20200701.csv', '20200701\\2/log_20200701.csv', '20200701\\3/log_20200701.csv']


In [6]:
def make_dataframe(file_path_1,file_path_2):
    df_1 = [pd.read_csv(file_path_1[k],encoding='cp932') for k in range(len(file_path_1))]
    df_2 = [pd.read_csv(file_path_2[k],encoding='cp932') for k in range(len(file_path_2))]
    df_2 = [k[k['件数'] != -20].reset_index() for k in df_2]
    df_2 = [k.loc[:,['開始','処理時間','件数']] for k in df_2]
    return df_1,df_2

In [7]:
df_1,df_2 = make_dataframe(file_path_1,file_path_2)

In [8]:
df_2

[              開始            処理時間   件数
 0  2020/7/1 0:02         0:00:00    0
 1  2020/7/1 0:06  0:02:27.097232   32
 2  2020/7/1 0:08  0:02:03.579021    0
 3  2020/7/1 0:10         0:00:00   37
 4  2020/7/1 0:12  0:00:14.062793   56
 5  2020/7/1 0:13  0:01:59.638201  100,
               開始            処理時間   件数
 0  2020/7/1 0:02         0:00:00    0
 1  2020/7/1 0:06  0:02:27.097232   32
 2  2020/7/1 0:08  0:02:03.579021    0
 3  2020/7/1 0:10         0:00:00   37
 4  2020/7/1 0:12  0:00:14.062793   56
 5  2020/7/1 0:13  0:01:59.638201  100,
               開始            処理時間   件数
 0  2020/7/1 0:02         0:00:00    0
 1  2020/7/1 0:06  0:02:27.097232   32
 2  2020/7/1 0:08  0:02:03.579021    0
 3  2020/7/1 0:10         0:00:00   37
 4  2020/7/1 0:12  0:00:14.062793   56
 5  2020/7/1 0:13  0:01:59.638201  100]

In [9]:
def make_file(df_1,df_2):
    df = pd.concat([df_2,df_1],axis=1)
    df['サイズ(MB)'] = df['サイズ(Byte)']/(1024**2)
    
    ii = [pd.to_datetime(i).time() for i in df_2['処理時間']]
    time_to_second = []
    for i in ii:
        if i.microsecond:
            s = '%06d' % i.microsecond
            if int(s[0]) >= 5:
                seconds = i.hour*3600 + i.minute*60+ i.second +1
                time_to_second.append(seconds)
            else:
                seconds = i.hour*3600 + i.minute*60+ i.second
                time_to_second.append(seconds)
        else:
            seconds = i.hour*3600 + i.minute*60+ i.second
            time_to_second.append(seconds)
    m,s = divmod(sum(time_to_second),60)
    h,m = divmod(m,60)
    sum_time='{0}:{1:02d}:{2:02d}'.format(h,m,s)
    
    sum_num = sum(df['件数'])
    sum_size = sum(df['サイズ(Byte)'])
    sum_size_MB = sum_size/(1024**2)
    
    df.loc['sum'] = ['',sum_time,sum_num,sum_size/1024**3,sum_size,sum_size_MB]
    
    return df

In [10]:
list_file = [make_file(df_1[i],df_2[i]) for i in range(len(df_1))]

In [11]:
list_file_1,list_file_2,list_file_3=[list_file[0],list_file[1],list_file[2]]

In [ ]:
def combine_to_excel(list_file_1,list_file_2,list_file_3):
    list_name = ['list_file_1','list_file_2','list_file_3']
    writer = pd.ExcelWriter('20200701.xlsx')
    for i in list_name:
        eval(i).to_excel(excel_writer = writer,sheet_name = i,index=False)
    writer.save()
    writer.close()

In [ ]:
combine_to_excel(list_file_1,list_file_2,list_file_3)